In [1]:
import pickle 
import numpy as np
import string 
import networkx  as nn 

with open("result/fb_result.pkl","rb+") as f:
    data_fb_res = pickle.loads(f.read()) 

refine_data = []
data = data_fb_res[0]
for data in data_fb_res:
    refine_res = list(filter(lambda x: "N"  in x, data[2]))
    refine_res = list(map(lambda x : x.split("\n")[1], refine_res))
    refine_res = list(map(lambda x: x.split(",")[1], refine_res))
    refine_data.append((data[1], refine_res))


In [2]:
import re
def get_edge(p:str):
    regex_node = re.compile("N[0-9]+")
    regex_edge = re.compile(":.")
    node = regex_node.search(p).group()  

    edge = regex_edge.search(p).group() if not regex_edge.search(p) is None else "" 
    if edge == "":
        return ([node])
    else:
         return (node, edge.replace(":","").strip())

def get_node(p:str):
    regex_node = re.compile("N[0-9]+")
    regex_edge = re.compile(":.")
    node = regex_node.search(p).group()  
    return node 

def parse_path_via_edge(path:str) -> list[str]:
    seq = path.replace("]","").replace("[", "").split("->")
    seq = list(map(lambda x: get_edge(x), seq))
    sequnce = [ ]
    for i in range(len(seq)):
        if len(seq[i+1]) == 1:
            sequnce.append((seq[i][0], seq[i][1], seq[i+1][0]))
            return sequnce
        else:
            sequnce.append((seq[i][0], seq[i][1], seq[i+1][0]))

def parse_path_seq(path:str) -> list[str]:
    seq = path.replace("]","").replace("[", "").split("->")
    seq = list(map(lambda x: get_node(x), seq))
    return seq


In [3]:
import networkx as nn 
graph = nn.DiGraph()
with open("data/facebook.qm", "r") as fb:
    file = fb.readlines()
    
    for line in file:
        if "->" in line:
            edge, ty = line.strip().split(":") 
            src, dst = edge.split("->")
            graph.add_edge(src.strip(), dst.strip())
        else:
            node, age, name, x, y , height, status = line.split(" ")
            node_name = node.split(":")[0]
            age_val = age.split(":")[1]
            name_val = name.split(":")[1]
            x_val = x.split(":")[1]
            y_val = y.split(":")[1]
            height_val = height.split(":")[1]
            status_val = status.split(":")[1].replace("\n","").replace('\"', "").strip() 
            graph.add_node(node_name, age= age_val, name = name_val, pos_x = x_val, pos_y = y_val, height = height_val, status = status_val)


In [4]:
### valid path exists in the Graph 
for expriment in refine_data:
    for path in expriment[1]:
        edge_seq = parse_path_via_edge(path)
        for edge in edge_seq:
            try:
                assert graph.has_edge(edge[0], edge[2])
            except AssertionError as _:
                print(expriment[0] + path)


In [5]:
### valid money, 
# there should be a model for p, q, such that each node satisfy p < age and q > age and q - p < 7
import z3.z3 as z3 
p = z3.Real("p") 
q = z3.Real("q")
money_query_data = refine_data[1][1]
wrong = 0
for path in money_query_data:
    path_seq = parse_path_seq(path)
    solver = z3.Solver()
    for node in path_seq:
        age = float(graph.nodes[node]["age"])
        solver.add(p< age)
        solver.add(q> age)
        solver.add(q-p < 7)
    try:
        assert solver.check()== z3.sat
    except AssertionError as _:
        wrong = wrong + 1
        print(path_seq)

In [6]:
### valid date query, only test the start point and the end point 
import z3.z3 as z3 
p = z3.Real("p") 
q = z3.Real("q")
money_query_data = refine_data[0][1]
wrong = 0
for path in money_query_data:
    path_seq = parse_path_seq(path)
    solver = z3.Solver()
    src_x = graph.nodes[path_seq[0]]["pos_x"]
    src_y = graph.nodes[path_seq[0]]["pos_y"]
    src_age = graph.nodes[path_seq[0]]["age"]
    dst_x = graph.nodes[path_seq[-1]]["pos_x"]
    dst_y = graph.nodes[path_seq[-1]]["pos_y"]
    dst_age = graph.nodes[path_seq[-1]]["age"]
    try:
        assert float(src_age) *0.5 + 7 <= float(dst_age) 
        assert abs(float(src_x) - float(dst_x)) + abs(float(src_y) - float(dst_y)) <= 20 
        assert graph.nodes[path_seq[-1]]["status"] == "single"
    except AssertionError as _:
        wrong = wrong + 1
        print(path_seq)

In [7]:
### valid central path query, validate that there is an x, y such that abs(x - node.x) + abs(y - node.y) <= 35
import z3.z3 as z3 
p = z3.Real("p") 
q = z3.Real("q")
money_query_data = refine_data[2][1]
wrong = 0
for path in money_query_data:
    path_seq = parse_path_seq(path)
    solver = z3.Solver()
    x = z3.Real("x")
    y = z3.Real("y")
    for node in path_seq:
        src_x = graph.nodes[node]["pos_x"]
        src_y = graph.nodes[node]["pos_y"]  
        solver.add(z3.Abs(x - src_x) + z3.Abs(y - src_y) <= 35)
    try:
        assert solver.check() == z3.sat
    except AssertionError as _:
        wrong = wrong + 1
        print(path_seq)

Validations for Youtube Graph 